In [ ]:
import pandas as pd

In [ ]:
history_folder = "./League-History"
results_folder = "./analysis_results"

# merge all the csv files from the subfolders into one dataframe, introducing a new column the folder and file name

df = pd.DataFrame()
for year in range(2015, 2023):
    for week in range(1, 18):
        try:
            df_temp = pd.read_csv(f'{history_folder}/{year}/{week}.csv')
            df_temp['year'] = year
            df_temp['week'] = week

            df = pd.concat([df, df_temp], ignore_index=True)

        except:
            pass

df

In [ ]:
# remove by weeks (where Opponent is '-')
df = df[df['Opponent'] != '-']

In [ ]:
# handle league-specific preprocessing
# comment out the line below if you don't have utils.league_utils
from utils.league_utils import LeagueUtils

df = LeagueUtils.preprocess_league(df)

In [ ]:
df['win'] = df['Total'] > df['Opponent Total']
df['loss'] = df['Total'] < df['Opponent Total']
df['tie'] = df['Total'] == df['Opponent Total']

In [ ]:
df.groupby(['year']).mean()['Total'].plot(kind='bar', title='Average Points Per Year')

In [ ]:
df.groupby(['week']).mean()['Total'].plot(kind='bar', title='Average Points Per Week')

In [ ]:
df[df['week'] > 2].groupby(['Owner'])['Rank'].mean().sort_values(ascending=False).plot(kind='barh', title='Average League Rank Per Week (after week 2)')

In [ ]:
owners_df = df.groupby(['Owner'])['Owner', 'win', 'loss', 'tie', 'Total', 'Opponent Total'].sum()

owners_df['games'] = owners_df['win'] + owners_df['loss'] + owners_df['tie']
number_of_games_threshold = 20
owners_df = owners_df[owners_df['games'] > number_of_games_threshold]

# calculate win percentage and points per game
owners_df['win_pct'] = owners_df['win'] / owners_df['games']
owners_df['ppg'] = owners_df['Total'] / owners_df['games']
owners_df['opp_ppg'] = owners_df['Opponent Total'] / owners_df['games']

# save to excel
owners_df.to_excel(f'{results_folder}/Owners.xlsx')
owners_df

Various plots about owner stats

In [ ]:
ax = owners_df['win_pct'].sort_values().plot(kind='barh', title='Win Percentage')
_ = ax.set_yticklabels(owners_df['win_pct'].sort_values().index)

In [ ]:
ax = owners_df['win'].sort_values().plot(kind='barh', title='Total Wins')
_ = ax.set_yticklabels(owners_df['win'].sort_values().index)

In [ ]:
# maybe skewed by half-PPR vs. non-PPR era
ax = owners_df['ppg'].sort_values().plot(kind='barh', title='Points Per Game')
_ = ax.set_yticklabels(owners_df['ppg'].sort_values().index)

In [ ]:
ax = owners_df['opp_ppg'].sort_values().plot(kind='barh', title='Opponent Points Per Game')
_ = ax.set_yticklabels(owners_df['opp_ppg'].sort_values().index)

In [ ]:
# get head-to-head matchup records
h2h = pd.DataFrame(df.groupby(['Owner', 'Opponent_Owner'])['Owner', 'Opponent_Owner', 'win', 'loss', 'tie', 'Total', 'Opponent Total'].sum()).reset_index()
h2h

In [ ]:
h2h['games'] = h2h['win'] + h2h['loss'] + h2h['tie']
h2h_number_of_games_threshold = 3
h2h = h2h[h2h['games'] > h2h_number_of_games_threshold]

h2h['win_pct'] = h2h['win'] / h2h['games']
h2h['win_diff'] = h2h['win'] - h2h['loss']
h2h['ppg'] = h2h['Total'] / h2h['games']
h2h['opp_ppg'] = h2h['Opponent Total'] / h2h['games']

# save to excel
h2h.to_excel(f'{results_folder}/Head-to-Head.xlsx')
h2h

Plot top x head-to-head matchups

In [ ]:
# filter top x win_pct and top 5 win_diff
h2h_win_pct = h2h.sort_values('win_pct', ascending=False).head(3)
h2h_win_diff = h2h.sort_values('win_diff', ascending=False).head(3)

# filter bottom 5 win_pct and top 5 win_diff
h2h_win_pct_l = h2h.sort_values('win_pct', ascending=True).head(3)
h2h_win_diff_l = h2h.sort_values('win_diff', ascending=True).head(3)

# concat the two dataframes without duplicates
h2h_top = pd.concat([h2h_win_pct, h2h_win_diff, h2h_win_pct_l, h2h_win_diff_l])
h2h_top = h2h_top.drop_duplicates()

# plot wins on x-axis, losses on y-axis
ax = h2h_top.plot(kind='scatter', x='loss', y='win', title='Wins vs. Losses', figsize=(10, 10))
_ = ax.set_xlabel('Losses')
_ = ax.set_ylabel('Wins')
_ = ax.set_xticks(range(0, 11))
_ = ax.set_yticks(range(0, 11))

# add labels to each point with random offset to avoid overlapping
import random
for i, row in h2h_top.iterrows():
    ax.annotate(row['Owner'] + ' vs. ' + row['Opponent_Owner'], (row['loss'], row['win'] + random.uniform(-0.3, 0.3)))


# show diagonal line
ax.plot([0, 10], [0, 10], color='red', linestyle='--', linewidth=1, alpha=0.5)


In [ ]:
h2h_top